In [1]:
import json
import nltk

In [2]:
with open("data/neurips_fulltexts", "r") as f:
    txt = f.readlines()

In [17]:
id_counter = 0
id_string = "nipsid_{}".format(id_counter)

missing_title = []

scirex_format_json = {}

for l in txt:
    id_counter += 1
    id_string = "nipsid_{}".format(id_counter)
    paper_dict = json.loads(l)
    local_json = {"words": [], "sentences": [], "sections": []}
    sentence_offset = 0
    
    if "title" in paper_dict["metadata"] and paper_dict["metadata"]["title"]:
        local_words = ["document", ":"] + nltk.word_tokenize(paper_dict["metadata"]["title"])
        if local_words:
            sentence_offset = len(local_words)
            local_json["sentences"].append([0, sentence_offset])
            local_json["words"] = local_words
        
    
    if "metadata" in paper_dict and "abstractText" in paper_dict["metadata"] and paper_dict["metadata"]["abstractText"]:
        try:
            abs_sents = nltk.sent_tokenize(paper_dict["metadata"]["abstractText"])
        except Exception as ex:
            print(ex)
#             print("On abs: ", paper_dict["metadata"]["abstractText"])
            abs_sents = []
        local_sec_words = []
        for i in abs_sents:
            abs_sent_words = nltk.word_tokenize(i)
            for w in abs_sent_words:
                local_sec_words.append(w)
            local_json["sentences"].append([sentence_offset, sentence_offset+len(abs_sent_words)])
            sentence_offset = sentence_offset+len(abs_sent_words)
    for w in local_sec_words:
        local_json["words"].append(w)
    local_json["sections"].append([0, len(local_json["words"])])
    
    if "metadata" in paper_dict and "sections" in paper_dict["metadata"] and paper_dict["metadata"]["sections"]:
        for sec in paper_dict["metadata"]["sections"]:
            local_sec_words = ["section", ":"]
            section_start_offset = sentence_offset 
            if "heading" in sec and sec["heading"]:
                sec_heading_words = sec["heading"].split(" ")
                local_sec_words += sec_heading_words
                local_json["sentences"].append([sentence_offset, sentence_offset+len(local_sec_words)])
                sentence_offset = sentence_offset + len(local_sec_words)
            if "text" in sec and sec["text"]:
                try:
                    section_text = nltk.sent_tokenize(sec["text"])
                    for ssent in section_text:
                        sec_sent_words = nltk.word_tokenize(ssent)
                        for w in sec_sent_words:
                            local_sec_words.append(w)
                        local_json["sentences"].append([sentence_offset, sentence_offset+len(sec_sent_words)])
                        sentence_offset = sentence_offset+len(sec_sent_words)
                except Exception as ex:
                    print(ex)
                    section_text = []
            section_end_offset = section_start_offset + len(local_sec_words)
            for w in local_sec_words:
                local_json["words"].append(w)
            local_json["sections"].append([section_start_offset, section_end_offset])
    scirex_format_json[id_counter] = {"doc_id": id_string, "sentences": local_json["sentences"], 
                                              "words": local_json["words"], "sections": local_json["sections"]}

In [19]:
id_counter, scirex_format_json[id_counter]["sections"]

(9671,
 [[0, 157],
  [157, 601],
  [601, 1175],
  [1175, 1187],
  [1187, 2089],
  [2089, 2588],
  [2588, 3042],
  [3042, 3227],
  [3227, 3488],
  [3488, 3550],
  [3550, 3630],
  [3630, 3734],
  [3734, 3793]])

In [24]:
scirex_format_json[id_counter]["words"][157:163], scirex_format_json[id_counter]["words"][601:606]

(['section', ':', '1', 'Introduction', 'Cortical', 'neural'],
 ['section', ':', '2', 'CCN', 'of'])

In [26]:
scirex_format_json[id_counter]["words"][157:161]

['section', ':', '1', 'Introduction']

In [29]:
scirex_format_json[id_counter]["sentences"][0:8]

[[0, 13],
 [13, 38],
 [38, 61],
 [61, 97],
 [97, 123],
 [123, 157],
 [157, 161],
 [161, 180]]

In [30]:
with open("scirex_neurips", "w") as fw:
    for v in scirex_format_json.values():
        fw.write(str(v))
        fw.write("\n")

In [31]:
!tail -1 scirex_neurips

{'doc_id': 'nipsid_9671', 'sentences': [[0, 13], [13, 38], [38, 61], [61, 97], [97, 123], [123, 157], [157, 161], [161, 180], [180, 200], [200, 255], [255, 299], [299, 316], [316, 348], [348, 385], [385, 404], [404, 427], [427, 467], [467, 522], [522, 539], [539, 563], [563, 587], [587, 601], [601, 609], [609, 643], [643, 667], [667, 688], [688, 725], [725, 842], [842, 852], [852, 879], [879, 924], [924, 943], [943, 944], [944, 945], [945, 975], [975, 999], [999, 1029], [1029, 1078], [1078, 1097], [1097, 1111], [1111, 1116], [1116, 1175], [1175, 1187], [1187, 1195], [1195, 1213], [1213, 1228], [1228, 1232], [1232, 1253], [1253, 1281], [1281, 1299], [1299, 1321], [1321, 1355], [1355, 1384], [1384, 1402], [1402, 1423], [1423, 1478], [1478, 1524], [1524, 1561], [1561, 1583], [1583, 1603], [1603, 1622], [1622, 1646], [1646, 1703], [1703, 1717], [1717, 1730], [1730, 1736], [1736, 1764], [1764, 1784], [1784, 1893], [1893, 1919], [1919, 1920], [1920, 1921], [1921, 1924], [1924, 1925], [1925, 

In [5]:
paper_dict.keys()

dict_keys(['name', 'metadata'])

In [7]:
paper_dict["metadata"]["sections"]

[{'heading': '1 Introduction',
  'text': 'Recent years have witnessed a wide range of machine learning and robust optimization applications being formulated as a min-max saddle point game; see [51, 11, 10, 50, 20, 53] and the references therein. Examples of problems that are formulated under this framework include generative adversarial networks (GANs) [51], reinforcement learning [11], adversarial learning [53], learning exponential families [10], fair statistical inference [17, 56, 52, 37], generative adversarial imitation learning [6, 27], distributed non-convex optimization [35] and many others. These applications require solving an optimization problem of the form\nmin θ∈Θ max α∈A f(θ,α). (1)\nThis optimization problem can be viewed as a zero-sum game between two players. The goal of the first player is to minimize f(θ,α) by tuning θ, while the other player’s objective is to maximize ∗Department of Industrial and Systems Engineering, University of Southern California †Data Science